In [3]:
# biblioteker

"""
bruker tensorflow nightly:

pip3 install -q tensorflow_text_nightly --user
pip3 install -q tf-nightly --user

"""

import os
import re
import shutil
import string
import tensorflow as tf

import tensorflow_text as text

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing


In [4]:
# importerer og blander dataen
batch_size = 32
seed = 42

# splitter treningsdataen i to, en til trening og en til validering av treningen
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "./data/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    './data/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    './data/test',
    batch_size=batch_size
)


Found 7649 files belonging to 7 classes.
Using 6120 files for training.
Found 7649 files belonging to 7 classes.
Using 1529 files for validation.
Found 1595 files belonging to 7 classes.


In [5]:

# skaffer vokabularet vi har laget

bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]
bert_vocab_args = dict(
    # maksimum størrelse for vokabularet
    vocab_size = 8000 * 7,
    # Reserverte orddeler som må være med
    reserved_tokens=reserved_tokens,
    # flere argumenter
    bert_tokenizer_params=bert_tokenizer_params,
    learn_params={},
)

# lager en "tokenizer", som deler tekst opp i orddeler
tokenizer = text.BertTokenizer('vocab.txt', **bert_tokenizer_params)



In [6]:
#funksjoner som blir brukt til å voktorisere teksten
# altså gjøre dem om til tall i stedet for bokstaver
# hvert tall er IDen til en orddel fra vokabularet vårt

# alle vektorene må være like lange, så vi legger til
# "Padding" på slutten av vektoren hvis den ikke er lang nok

# IDen til padding
PAD_ID = 0
# Maksimum lengde for vektoren
# hvis vektoren er mindre, blir det lagt til padding
max_seq_len = 20

# denne funskjonen er her for å passe formatet vi skal bruke den i senere
def vectorize(text, label):
  # text er tekst inputtet, og label er hvilket språk det er på
  ids, mask, type_ids = preprocess_bert_input(text)
  return (ids, mask, type_ids), label

def preprocess_bert_input(text):
  # finner IDene til alle orddelene i inputtet
  ids = tokenize_text(text, max_seq_len)
  # lager en mask, som i dette tilfettet representerer lengden på vektoren vår
  mask = tf.cast(ids > 0, tf.int64)
  mask = tf.reshape(mask, [-1, max_seq_len])
  # lager den ferdige vektoren
  # først fyller lager vi en vektor med
  # den riktige lengden (shape) fyllt med nuller
  zeros_dims = tf.stack(tf.shape(mask))
  type_ids = tf.fill(zeros_dims, PAD_ID)
  # så setter vi inn de faktiske orddelenes IDer
  type_ids = tf.cast(type_ids, tf.int64)

  return (ids, mask, type_ids)

def tokenize_text(text, seq_len):
  # bruker "tokenizeren" vi lagde tidligere til å generere tokens som passer teksten
  tokens = tokenizer.tokenize(text)
  # tilpasser outputtet
  tokens = tokens.merge_dims(1, 2)[:, :seq_len]

  # klipper vekk slutten hvis den er lenger enn maksimum lengde
  tokens = tokens[:, :seq_len]
  # legger til padding hvis den er kortere enn maksimum lengde
  tokens = tokens.to_tensor(default_value=PAD_ID)
  pad = seq_len - tf.shape(tokens)[1]
  tokens = tf.pad(tokens, [[0, 0], [0, pad]], constant_values=PAD_ID)
  return tf.reshape(tokens, [-1, seq_len])

In [7]:
# vektoriserer hvert datasett, med funksjonen vi lagde med det spesielle formatet
train_ds = raw_train_ds.map(vectorize)
val_ds = raw_val_ds.map(vectorize)
test_ds = raw_test_ds.map(vectorize)


In [8]:
# printer en bit av dataen for å se at det fungerer
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Tekst: ", first_review)
print("\nSpråk: ", raw_train_ds.class_names[first_label])
vocab = [w.strip() for w in open("vocab.txt", encoding="utf-8").readlines()]
print("\nOrddeler: ", 
    " ".join(vocab[id] for id in vectorize(first_review, first_label)[0][0][0])
)


Tekst:  tf.Tensor(b'turkiet aer en viktig regional makt i denna del av eurasien med kulturellt och ekonomiskt inflytande i omraadet mellan adriatiska havet i vaester och kina i oester ryssland i norr och mellanoestern i soeder och har kommit att faa en oekande strategisk betydelse18 procent av turkiets befolkning beraeknas vara kurder men siffran har aeven uppskattats vara runt 20 procent', shape=(), dtype=string)

Språk:  sv

Orddeler:  turkiet a ##er en viktig regional makt i denna del av eu ##ras ##ien med kultur ##ellt och e ##konomi


In [9]:
# optimaliserer dataen (er ærlig talt ikke sikker på hva dette gjør men alle andre prosjekter gjør det)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [10]:
# designer modellen som skal bli lært


model = tf.keras.Sequential([
  # omformer inputtet til tall som er "bedre å lære med"
  # i stedet for et heltall som ID for hver orddel, blir det til en liste
  # med nuller for hvert språk det ikke er og en 1 for det språket som er riktig
  # dette gjør at modellen ikke tror at orddeler som har tall nærme hverandre likner hverandre
  layers.Embedding(10000, 32, input_length=max_seq_len),
  # del av netverket som blir trent
  layers.Dropout(0.2),
  # passer på at dataen forstsatt har lik lengde
  layers.GlobalAveragePooling1D(),
  # del av netverket som blir trent
  layers.Dropout(0.2),
  # output (en for hvert språk, som det er 7 av)
  layers.Dense(7)])

# printer ut modellen
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 32)            320000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 32)            0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 7)                 231       
Total params: 320,231
Trainable params: 320,231
Non-trainable params: 0
_________________________________________________________________


In [11]:
# kompilerer modellen
# optimisereren er en funksjon som proøver å forbedre modellen
# i hver iterasjon av læringen
optimizer = tf.keras.optimizers.Adam()

model.compile(
    # loss funksjonen beregner hvor langt unna det riktige svaret modellen er
    loss=losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer=optimizer, metrics = ["accuracy"]
)


In [13]:
epochs = 30
# trener modellen på datasettet, 50 ganger
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/30
192/192 [==============================] - 1s 6ms/step - loss: 0.0539 - accuracy: 0.9931 - val_loss: 0.1674 - val_accuracy: 0.9372
Epoch 2/30
192/192 [==============================] - 1s 6ms/step - loss: 0.0514 - accuracy: 0.9925 - val_loss: 0.1676 - val_accuracy: 0.9379
Epoch 3/30
192/192 [==============================] - 1s 5ms/step - loss: 0.0459 - accuracy: 0.9941 - val_loss: 0.1664 - val_accuracy: 0.9366
Epoch 4/30
192/192 [==============================] - 1s 6ms/step - loss: 0.0420 - accuracy: 0.9954 - val_loss: 0.1675 - val_accuracy: 0.9359
Epoch 5/30
192/192 [==============================] - 1s 6ms/step - loss: 0.0401 - accuracy: 0.9956 - val_loss: 0.1675 - val_accuracy: 0.9353
Epoch 6/30
192/192 [==============================] - 1s 6ms/step - loss: 0.0350 - accuracy: 0.9972 - val_loss: 0.1686 - val_accuracy: 0.9353
Epoch 7/30
192/192 [==============================] - 1s 6ms/step - loss: 0.0346 - accuracy: 0.9966 - val_loss: 0.1684 - val_accuracy: 0.9353
Epoch 

In [14]:
# tester hvor bra modellen er med test datasettet
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


50/50 [==============================] - 3s 45ms/step - loss: 0.2323 - accuracy: 0.9279
Loss:  0.23226961493492126
Accuracy:  0.9278996586799622


In [15]:

# eksporterer den ferdigtrente modellen så vi kan sette inn våre egne input
export_model = tf.keras.Sequential([
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)


In [18]:
# nå kan vi teste egne input!
import unidecode
inp = input("> ")

# fjern linjeskift
inp = inp.replace('\n', ' ').lower()
# erstatt æøå osv
inp = inp.replace('æ', 'ae').replace(
    'å', 'aa').replace('ø', 'oe').replace('ö', 'oe').replace('ä', 'ae').replace('ä', 'ae')
inp = unidecode.unidecode(inp)
# fjern noen tall wikipedia setter inn for kilder eller noe
inp = re.sub(r"\[[0-9]*\]", "", inp)
# fjern tegnsetting
inp = inp.translate(str.maketrans('', '', string.punctuation))
# deler inputtet inn i orddeler
tok = preprocess_bert_input(inp)
# kjører modellen på inputtet
result = export_model.predict([tok])[0]

# printer resultatet
best_index = 0
best_score = 0
for i in range(len(result)):
    print(f"{raw_train_ds.class_names[i]}: {result[i]}")
    if result[i] > best_score:
        best_score = result[i]
        best_index = i
print("Result: " + raw_train_ds.class_names[best_index])

da: 0.867804765701294
en: 0.06320694088935852
es: 0.353193461894989
ja: 0.1558380424976349
nb: 0.9753385782241821
nn: 0.08760064840316772
sv: 0.07165762782096863
Result: nb


In [19]:
print(raw_train_ds.class_names)

['da', 'en', 'es', 'ja', 'nb', 'nn', 'sv']


In [20]:
export_model.save("sprakmodell.h5")